In [1]:
import os
import pandas as pd 

In [3]:
df = pd.read_csv('t16.csv', sep = ',')
df['jjmmaa'] = pd.to_datetime(df['jjmmaa'])  

In [4]:
df.head(3)

,rgs,pr,m,js,jjmmaa,hhmm,Vitesse,To,Qualite,Q,A,Code
0,101#M1a,0,430,Ve,2016-01-01,00:00,92.0,1.0,?,421.0,101,M1a
1,102#M1b,1,9,Ve,2016-01-01,00:00,92.0,3.0,,835.0,102,M1b
2,103#M1c,1,508,Ve,2016-01-01,00:00,0.0,0.0,,0.0,103,M1c


In [5]:
vacances_hiver = pd.date_range('2016-02-06','2016-02-26')
vacances_nouvelan = pd.date_range('2016-01-01','2016-01-04')
vacances_printemps = pd.date_range('2016-04-02','2016-04-18')
vacances_ptascenssion = pd.date_range('2016-05-04','2016-05-09')
vacances_ete = pd.date_range('2016-07-05','2016-09-01')

In [6]:
df1 = df.loc[~df['jjmmaa'].isin(vacances_hiver),:]
df2 = df1.loc[~df1['jjmmaa'].isin(vacances_nouvelan),:]
df3 = df2.loc[~df2['jjmmaa'].isin(vacances_printemps),:]
df4 = df3.loc[~df3['jjmmaa'].isin(vacances_ptascenssion),:]
df5 = df4.loc[~df4['jjmmaa'].isin(vacances_ete),:]

In [7]:
df = df5

In [8]:
df.head(3)

,rgs,pr,m,js,jjmmaa,hhmm,Vitesse,To,Qualite,Q,A,Code
5496,101#M1a,0,430,Sa,2016-02-01,00:00,94.0,1.0,?,351.0,101,M1a
5497,102#M1b,1,9,Sa,2016-02-01,00:00,95.0,3.0,,845.0,102,M1b
5498,103#M1c,1,508,Sa,2016-02-01,00:00,0.0,0.0,,0.0,103,M1c


In [9]:
# Garder que les Mardis et les Jeudis
df1 = df[df['js'].str.startswith('Ma')]
df2 = df[df['js'].str.startswith('Je')]

In [14]:
df = pd.concat([df1, df2])

In [17]:
df.index = range(len(df))

In [6]:
# Pour enlever les weekends
#df = df[~df['js'].astype(str).str.startswith('Sa')]
#df = df[~df['js'].astype(str).str.startswith('Di')]

In [18]:
# L'heure de pointe du matin de 7.30 a 9h
cond1 = df.apply ( lambda row : row["hhmm"] > '07:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '09:06', axis = 1 )
# L'intersection des deux condition.
df1 = pd.merge(df[cond1], df[cond2], how='inner')

In [19]:
# de 9h a 10h
cond1 = df.apply ( lambda row : row["hhmm"] > '08:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '10:06', axis = 1 )
# L'intersection des deux condition.
df2 = pd.merge(df[cond1], df[cond2], how='inner')

In [20]:
# L'heure de pointe du matin
cond1 = df.apply ( lambda row : row["hhmm"] > '11:54', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '13:06', axis = 1 )
# L'intersection des deux condition.
df3 = pd.merge(df[cond1], df[cond2], how='inner')

In [21]:
# L'heure de pointe du Soir de 17 a 19
cond1 = df.apply ( lambda row : row["hhmm"] > '17:00', axis = 1 )
cond2 = df.apply ( lambda row : row["hhmm"] < '19:06', axis = 1 )
# L'intersection des deux condition.
df4 = pd.merge(df[cond1], df[cond2], how='inner')

## La moyenne de la vitesse et le debit des heures de pointes

In [22]:
vitesse_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').mean()
vitesse_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').mean()

In [23]:
VQ = pd.concat([vitesse_8_9,vitesse_9_10,vitesse_12_13,vitesse_17_19], axis = 1)
VQ.columns = ['V_8_9','Q_8_9','V_9_10','Q_9_10','V_12_13','Q_12_13','V_17_19','Q_17_19']

In [24]:
# Reindexer la table
VQ['Code'] = VQ.index
VQ.index = range(len(VQ))

In [25]:
# table isidor
df_isidor = pd.read_csv("table3.csv",header=None, sep = ';')
# renommer la colonne 
df_isidor.columns = ['Code']
# renommer qlq stations
df_isidor['Code'] = df_isidor['Code'].str.replace('MM713.','M7')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM613.','M6')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM513.','M5')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM413.','M4')
df_isidor['Code'] = df_isidor['Code'].str.replace('MM113.','M1')

In [26]:
#L'union Left
result = pd.merge(df_isidor, VQ, how = 'left')

In [27]:
len(result) == len(df_isidor)

True

In [28]:
# Exporter la table
result.to_csv('VQ.csv', index = False, encoding = 'utf-8')

## Les déciles 8 et 9 des vitesse des heures de pointe

In [29]:
decile8_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile8_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile8_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile8_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)

decile9_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile9_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile9_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile9_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)

In [31]:
VQ_deciles = pd.concat([decile8_8_9,decile8_9_10,decile8_12_13,decile8_17_19,decile9_8_9,decile9_9_10
                        ,decile9_12_13,decile9_17_19], axis = 1)
VQ_deciles.columns = ['V8_8_9','Q8_8_9','V8_9_10','Q8_9_10','V8_12_13','Q8_12_13','V8_17_19','Q8_17_19'
                      ,'V9_8_9','Q9_8_9','V9_9_10','Q9_9_10','V9_12_13','Q9_12_13','V9_17_19','Q9_17_19']

In [32]:
VQ_deciles.head()

,V8_8_9,Q8_8_9,V8_9_10,Q8_9_10,V8_12_13,Q8_12_13,V8_17_19,Q8_17_19,V9_8_9,Q9_8_9,V9_9_10,Q9_9_10,V9_12_13,Q9_12_13,V9_17_19,Q9_17_19
Code,,,,,,,,,,,,,,,,
M1A,2997.4,92.0,2799.4,94.0,2256.4,95.0,3291.8,92.0,3103.1,93.0,2924.4,95.0,2304.8,96.0,3410.0,93.0
M1B,2983.8,89.0,2896.2,91.0,2289.8,93.0,3340.6,88.0,3047.7,89.7,2977.2,92.0,2330.0,93.0,3448.8,88.4
M1C,5316.0,89.0,5195.8,91.0,4039.4,93.0,5969.8,87.0,5408.1,89.0,5316.4,92.0,4107.4,94.0,6163.8,88.0
M1D,5127.4,92.0,4990.2,94.0,3896.4,96.0,5892.8,89.0,5257.9,93.0,5225.6,95.0,3952.6,96.0,6030.6,91.0
M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Reindexer la table
VQ_deciles['Code'] = VQ_deciles.index
VQ_deciles.index = range(len(VQ_deciles))

In [34]:
#L'union Left
result_deciles = pd.merge(df_isidor, VQ_deciles, how = 'left')

In [35]:
len(result_deciles) == len(df_isidor)

True

In [36]:
# Exporter la table
result_deciles.to_csv('VQ_deciles.csv', index = False, encoding = 'utf-8')

## Les déciles 1 et 2 des vitesse des heures de pointe

In [38]:
decile1_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile1_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile1_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)
decile1_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.8)

decile2_8_9 = df1[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile2_9_10 = df2[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile2_12_13 = df3[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)
decile2_17_19 = df4[['Code','Vitesse','Q']].groupby('Code').quantile(0.9)

In [39]:
VQ_deciles12 = pd.concat([decile1_8_9,decile1_9_10,decile1_12_13,decile1_17_19,decile2_8_9,decile2_9_10
                        ,decile2_12_13,decile2_17_19], axis = 1)
VQ_deciles12.columns = ['V1_8_9','Q1_8_9','V1_9_10','Q1_9_10','V1_12_13','Q1_12_13','V1_17_19','Q1_17_19'
                      ,'V2_8_9','Q2_8_9','V2_9_10','Q2_9_10','V2_12_13','Q2_12_13','V2_17_19','Q2_17_19']

In [41]:
# Reindexer la table
VQ_deciles12['Code'] = VQ_deciles12.index
VQ_deciles12.index = range(len(VQ_deciles12))

In [42]:
#L'union Left
result_deciles12 = pd.merge(df_isidor, VQ_deciles12, how = 'left')

In [43]:
# Exporter la table
result_deciles12.to_csv('VQ_deciles12.csv', index = False, encoding = 'utf-8')